# Imports

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, kendalltau
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import re
import string
import nltk
import random
# import libvoikko

C:\Users\utente\anaconda3\envs\tm\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\utente\anaconda3\envs\tm\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\utente\anaconda3\envs\tm\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Data Importing and Sorting

In [2]:
files = os.listdir('corpus')
files.remove('.DS_Store')
files.remove('scores_ru-en.csv')
scaler = MinMaxScaler()
for file_ in files:
  name = file_.split('-')[0] + file_.split('-')[1]
  vars()[name] = pd.read_csv(os.path.join('corpus', file_, 'scores.csv'))
  vars()[name].drop(columns = ['source', 'annotators', 'z-score'], inplace = True)
  vars()[name]['avg-score'] = scaler.fit_transform(vars()[name]['avg-score'].values.reshape(-1,1)) #normalizing values betwewen 0 and 1

In [3]:
english = csen.copy()
for df in [deen, ruen, zhen]:
  english = english.append(df)

In [4]:
finnish = enfi.copy()
chinese = enzh.copy()

# Train, Dev & Test Split

In [5]:
en_train, en_dev = train_test_split(english, shuffle = True, test_size = 0.2, random_state = 7)
en_dev, en_test = train_test_split(en_dev, shuffle = True, test_size = 0.5, random_state = 7)

fin_train, fin_dev = train_test_split(finnish, shuffle = True, test_size = 0.2, random_state = 7)
fin_dev, fin_test = train_test_split(fin_dev, shuffle = True, test_size = 0.5, random_state = 7)

ch_train, ch_dev = train_test_split(chinese, shuffle = True, test_size = 0.2, random_state = 7)
ch_dev, ch_test = train_test_split(ch_dev, shuffle = True, test_size = 0.5, random_state = 7)

# Cleaning Function

In [68]:
def clean(text_list,
          command_dict = None,
#           lower = False,
#           keep_numbers = False,
#           keep_expression = False,
#           remove_char = False,
#           remove_stop = False,
#           remove_tag = False,
#           lemmatize = False,
#           stemmer = False,
          english = True
          ):
    """
    Function that a receives a list of strings and preprocesses it.
    
    :param text_list: List of strings.
    :param lemmatize: Tag to apply lemmatization if True.
    :param stemmer: Tag to apply the stemmer if True.
    """
    
    if english:
        lang = 'english'
    else:
        lang = 'finnish'
    
    stop = set(stopwords.words(lang))
    stem = SnowballStemmer(lang)
    
    updates = []
    for j in tqdm(range(len(text_list))):
        
        text = text_list[j]
        
        #LOWERCASE TEXT
        if command_dict['lower']:
            text = text.lower()
            
        #KEEP NUMBERS AS TOKENS
        if command_dict['keep_numbers']:
            text = re.sub("[\d+]", 'NUMBER', text)
        
        #KEEP '?' and '!' AS TOKENS
        if command_dict['keep_expression']:
            text = re.sub("[\?|\!]", 'EXPRESSION', text)
            
        #REMOVE THAT IS NOT TEXT
        if command_dict['remove_char']:
            text = re.sub("[^a-zA-Z]", ' ', text)
            
        #REMOVE TAGS
        if command_dict['remove_tag']:
            text = BeautifulSoup(text).get_text()
        
        #REMOVE STOP WORDS
        if command_dict['remove_stop']:
            text = ' '.join([word for word in text.split(' ') if word not in stop])
        
        #LEMMATIZATION
        if command_dict['lemmatize']:
            if english:
                lemma = WordNetLemmatizer()
                text = " ".join(lemma.lemmatize(word) for word in text.split())
#             else:
#                 lemma = libvoikko.Voikko(u"fi")
#                 text = " ".join(lemma.analyze(word)[0]['BASEFORM'] for word in text.split())
        
        #STEMMER
        if command_dict['stemmer']:
            text = " ".join(stem.stem(word) for word in text.split())
        
        updates.append(text)
        
    return updates

def clean_zh_stopwords(text_list, stopwords_set='merged'):
    """
    Function that removes chinese stopwords
    
    :param stopwords_set: remove words of both sets (merged), just the 1st (fst) or just the second (snd) 
    """
    updates = []
    
    zh_stopwords1 = [line.strip() for line in open('chinese_stopwords/chinese_stopwords1.txt', 'r', encoding='utf-8').readlines()]
    zh_stopwords2 = [line.strip() for line in open('chinese_stopwords/chinese_stopwords2.txt', 'r', encoding='utf-8').readlines()]
    
    if stopwords_set == 'merged':
        stop = list(set(zh_stopwords1 + zh_stopwords2))
    elif stopwords_set == 'fst':
        stop = zh_stopwords1
    elif stopwords_set == 'snd':
        stop = zh_stopwords2
        

    for j in tqdm(range(len(text_list))):
        text = text_list[j]
        text = ' '.join([word for word in text.split(' ') if word not in stop])
        updates.append(text)
        
    return updates
        
    
def update_df(dataframe, list_updated):
    dataframe.update(pd.DataFrame({"Text": list_updated}))

# Baseline encoding

In [7]:
baseline_encoder_reference = CountVectorizer(max_features =  15000)#settting limit for computational reasons
# baseline_encoder_translation = CountVectorizer(max_features = 5000)

# Correlation Function

In [8]:
def cos_sim(reference, translation):
    
    cos = []
    for i in range(reference.shape[0]):
        cos.append(cosine_similarity(reference[i], translation[i])[0])
    cos = np.array(cos)
    cos.shape = (cos.shape[0],)
    
    return cos

In [9]:
def correlation(y_train, pred_train, y_dev, pred_dev, y_test, pred_test, print = False):

    corr_train, corr_train_pvalue = pearsonr(y_train, pred_train)
    corr_dev, corr_dev_pvalue = pearsonr(y_dev, pred_dev)
    corr_test, corr_test_pvalue = pearsonr(y_test, pred_test)

    corr_ktau_train, corr_ktau_train_pvalue = kendalltau(y_train, pred_train)
    corr_ktau_dev, corr_ktau_dev_pvalue = kendalltau(y_dev, pred_dev)
    corr_ktau_test, corr_ktau_test_pvalue = kendalltau(y_test, pred_test)
    
    if print:
        print(f'Pearson correlation between cosine similarity and score on training set: {corr_train} (p-value < 0.001: {corr_train_pvalue < 0.001}); and Kendall Tau: {corr_ktau_train} (p-value < 0.001: {corr_ktau_train_pvalue < 0.001})')
        print(f'Pearson correlation between cosine similarity and score on development set: {corr_dev} (p-value < 0.001: {corr_dev_pvalue < 0.001}); and Kendall Tau: {corr_ktau_dev} (p-value < 0.001: {corr_ktau_dev_pvalue < 0.001})')
        print(f'Pearson correlation between cosine similarity and score on test set: {corr_test} (p-value < 0.001: {corr_test_pvalue < 0.001}); and Kendall Tau: {corr_ktau_test} (p-value < 0.001: {corr_ktau_test_pvalue < 0.001})')

    return corr_train, corr_dev, corr_test, corr_ktau_train, corr_ktau_dev, corr_ktau_test

# Randomized search in cleaning methods

In [10]:
def random_cleaning_config(previuous_config):
    
    while True:
        random_cleaning_paramspace = {'lower' : random.choice([False, True]),
                                      'keep_numbers' : random.choice([False, True]),
                                      'keep_expression' : random.choice([False, True]),
                                      'remove_char' : random.choice([False, True]),
                                      'remove_stop' : random.choice([False, True]),
                                      'remove_tag' : random.choice([False, True]),
                                      'lemmatize' : random.choice([False, True]),
                                      'stemmer' : random.choice([False, True])}
            
        if random_cleaning_paramspace not in previuous_config:
            break
    
    return random_cleaning_paramspace

In [ ]:
types = ['train', 'dev', 'test']
results = {}
for iteration in tqdm(range(100)):

    cleaning_config = random_cleaning_config(list(results.keys()))
   
    for i,df in enumerate([en_train, en_dev, en_test]):
        
        df_noindex = df.reset_index(drop = True).copy()
        
        cleaned_df = pd.DataFrame()

        for column in ['reference', 'translation']:

            cleaned_df[column] = clean(df_noindex[column], cleaning_config)


            if i == 0 and column == 'reference':
                encoded_reference= baseline_encoder_reference.fit_transform(cleaned_df[column]).todense()

            elif column == 'reference':
                encoded_reference = baseline_encoder_reference.transform(cleaned_df[column]).todense()
            else:
                encoded_translation = baseline_encoder_reference.transform(cleaned_df[column]).todense()

        cos_name = 'cos_sim_' + types[i]
        vars()[cos_name] = cos_sim(encoded_reference, encoded_translation)


    results[str(cleaning_config)] = (correlation(cos_sim_train, en_train['avg-score'], cos_sim_dev, en_dev['avg-score'], cos_sim_test, en_test['avg-score']))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/62150 [00:00<?, ?it/s]

  0%|          | 0/62150 [00:00<?, ?it/s]

  0%|          | 0/7769 [00:00<?, ?it/s]

  0%|          | 0/7769 [00:00<?, ?it/s]

...

  0%|          | 0/7769 [00:00<?, ?it/s]

  0%|          | 0/7769 [00:00<?, ?it/s]

  0%|          | 0/7769 [00:00<?, ?it/s]

  0%|          | 0/62150 [00:00<?, ?it/s]

  0%|          | 0/62150 [00:00<?, ?it/s]

# Graphical Visualization of results

In [16]:
results

{"{'lower': True, 'keep_numbers': False, 'keep_expression': False, 'remove_char': True, 'remove_stop': False, 'remove_tag': True, 'lemmatize': True, 'stemmer': False}": (0.2648266398166333,
  0.2707478173452867,
  0.2643947978975451,
  0.1812399742083328,
  0.18168627091458345,
  0.1806681219213875),
 "{'lower': False, 'keep_numbers': False, 'keep_expression': False, 'remove_char': True, 'remove_stop': True, 'remove_tag': False, 'lemmatize': False, 'stemmer': True}": (0.2648266398166333,
  0.2707478173452867,
  0.2643947978975451,
  0.1812399742083328,
  0.18168627091458345,
  0.1806681219213875),
 "{'lower': True, 'keep_numbers': False, 'keep_expression': False, 'remove_char': True, 'remove_stop': False, 'remove_tag': False, 'lemmatize': False, 'stemmer': True}": (0.2648266398166333,
  0.2707478173452867,
  0.2643947978975451,
  0.1812399742083328,
  0.18168627091458345,
  0.1806681219213875),
 "{'lower': False, 'keep_numbers': False, 'keep_expression': True, 'remove_char': False, 're

In [12]:
results_df = pd.DataFrame(results).T.set_axis(['pearson_train', 'pearson_dev', 'pearson_test', 'ktau_train', 'ktau_dev', 'ktau_test'], axis = 1)

In [13]:
results_df.head()

,pearson_train,pearson_dev,pearson_test,ktau_train,ktau_dev,ktau_test
"{'lower': True, 'keep_numbers': False, 'keep_expression': False, 'remove_char': True, 'remove_stop': False, 'remove_tag': True, 'lemmatize': True, 'stemmer': False}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': False, 'keep_numbers': False, 'keep_expression': False, 'remove_char': True, 'remove_stop': True, 'remove_tag': False, 'lemmatize': False, 'stemmer': True}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': True, 'keep_numbers': False, 'keep_expression': False, 'remove_char': True, 'remove_stop': False, 'remove_tag': False, 'lemmatize': False, 'stemmer': True}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': False, 'keep_numbers': False, 'keep_expression': True, 'remove_char': False, 'remove_stop': False, 'remove_tag': False, 'lemmatize': False, 'stemmer': False}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': False, 'keep_numbers': True, 'keep_expression': True, 'remove_char': False, 'remove_stop': True, 'remove_tag': False, 'lemmatize': False, 'stemmer': True}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668


In [14]:
results_df.to_csv('randomized_search.csv')

In [23]:
config

{'lower': False,
 'keep_numbers': False,
 'keep_expression': False,
 'remove_char': True,
 'remove_stop': True,
 'remove_tag': False,
 'lemmatize': True,
 'stemmer': True}

In [15]:
results_df.sort_values('pearson_train')

,pearson_train,pearson_dev,pearson_test,ktau_train,ktau_dev,ktau_test
"{'lower': True, 'keep_numbers': False, 'keep_expression': False, 'remove_char': True, 'remove_stop': False, 'remove_tag': True, 'lemmatize': True, 'stemmer': False}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': True, 'keep_numbers': False, 'keep_expression': False, 'remove_char': True, 'remove_stop': True, 'remove_tag': True, 'lemmatize': True, 'stemmer': False}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': False, 'keep_numbers': False, 'keep_expression': True, 'remove_char': False, 'remove_stop': False, 'remove_tag': False, 'lemmatize': True, 'stemmer': True}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': True, 'keep_numbers': True, 'keep_expression': True, 'remove_char': False, 'remove_stop': False, 'remove_tag': False, 'lemmatize': False, 'stemmer': False}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': False, 'keep_numbers': False, 'keep_expression': False, 'remove_char': False, 'remove_stop': True, 'remove_tag': True, 'lemmatize': False, 'stemmer': True}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
...,...,...,...,...,...,...
"{'lower': True, 'keep_numbers': False, 'keep_expression': True, 'remove_char': False, 'remove_stop': False, 'remove_tag': True, 'lemmatize': True, 'stemmer': False}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': True, 'keep_numbers': False, 'keep_expression': False, 'remove_char': False, 'remove_stop': False, 'remove_tag': False, 'lemmatize': True, 'stemmer': False}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': True, 'keep_numbers': True, 'keep_expression': False, 'remove_char': False, 'remove_stop': False, 'remove_tag': False, 'lemmatize': False, 'stemmer': False}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668
"{'lower': False, 'keep_numbers': True, 'keep_expression': True, 'remove_char': True, 'remove_stop': True, 'remove_tag': True, 'lemmatize': False, 'stemmer': True}",0.264827,0.270748,0.264395,0.18124,0.181686,0.180668


In [70]:
cleaned.head()

,reference,cleaned
0,I was hoping to be in it until the end.,i hope end
1,The Imams interviewed by Kommersant who conduc...,the imam interview kommers conduct sermon russ...
2,Fiji gave master-class on the ownership of bal...,fiji gave master class ownership ball general ...
3,"He did not violate his information duties, he ...",he violat inform duti said
4,"An investigation has been opened, which is a ""...",an investig open logic step situat gill hofste...


In [27]:
cleaned.head()

,reference,cleaned
0,I was hoping to be in it until the end.,i hope end
1,The Imams interviewed by Kommersant who conduc...,the imam interview kommers conduct sermon russ...
2,Fiji gave master-class on the ownership of bal...,fiji gave master class ownership ball general ...
3,"He did not violate his information duties, he ...",he violat inform duti said
4,"An investigation has been opened, which is a ""...",an investig open logic step situat gill hofste...
